<a href="https://colab.research.google.com/github/Kiron-Ang/DSC/blob/main/web_scraping_schedule_of_classes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
!python -V

import requests

prefix_list = ["BINF", "BME", "CSI", "CSF", "DSC", "ELC", "EGR", "ME"]

for prefix in prefix_list:
  url = "https://www1.baylor.edu/scheduleofclasses/Results.aspx?"
  url += f"Term=202510&College=Z&Prefix={prefix}&StartCN=Z&EndCN=Z&"
  url += "Status=Z&Days=Z&Instructor=&IsMini=false&OnlineOnly=0&"
  url += "POTerm=Z&CourseAttr=Z&Sort=SN"

  response = requests.get(url)

  html_content = response.text

  html_content = html_content.splitlines()

  for line in html_content:
    if "col-md-2" in line:
      print(line)

Python 3.10.12
                        <div class="col-md-2"><strong>BINF 3360</strong></div>
                        <div class="col-md-2"><strong>BINF 43C9</strong></div>
                        <div class="col-md-2"><strong>BME 4360</strong></div>
                        <div class="col-md-2"><strong>BME 4372</strong></div>
                        <div class="col-md-2"><strong>BME 4374</strong></div>
                        <div class="col-md-2"><strong>BME 5360</strong></div>
                        <div class="col-md-2"><strong>BME 5V99</strong></div>
                        <div class="col-md-2"><strong>CSI 1401</strong></div>
                        <div class="col-md-2"><strong>CSI 1402</strong></div>
                        <div class="col-md-2"><strong>CSI 1430</strong></div>
                        <div class="col-md-2"><strong>CSI 1440</strong></div>
                        <div class="col-md-2"><strong>CSI 2300</strong></div>
                        <div class="col-md-2"><